In [1]:
import pandas as pd

In [89]:
inFPath = 'budget2021.xlsx'
df = pd.read_excel(inFPath, sheet_name='Sheet1', nrows=13)

In [90]:
cols = [c for c in df.columns if 'unnamed' not in c.lower()]
df = df[cols]
df

,Y1 Months,Y1 Requested salary,Y1 Fringe benefits,Y1 Total Effort,Y1 Material,Y1 Travel,Y1 Directs,Y1 Indirects,Y1 Total Requested funds,Y2 Months,...,Y4 Total Requested funds,Y5 Months,Y5 Requested salary,Y5 Fringe benefits,Y5 Total Effort,Y5 Material,Y5 Travel,Y5 Directs,Y5 Indirects,Y5 Total Requested funds
0,9.0,136246.15,40873.85,177120.0,NaN,NaN,NaN,NaN,NaN,7.0,...,NaN,6.0,117001.54,35100.46,152102.0,NaN,NaN,NaN,NaN,NaN
1,12.5,77777.51,19055.49,96833.0,NaN,NaN,NaN,NaN,NaN,18.0,...,NaN,12.0,87131.73,21347.27,108479.0,NaN,NaN,NaN,NaN,NaN
2,21.5,214023.66,59929.34,273953.0,NaN,NaN,NaN,NaN,NaN,25.0,...,NaN,18.0,204133.27,56447.73,260581.0,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,5400.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN
5,NaN,NaN,NaN,442421.0,NaN,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,442421.0,NaN,4500.0,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,6000.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,12000.0,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,6000.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,16500.0,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,285353.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,277081.0,NaN,NaN


In [184]:
# Personnel budget
personnel = df.iloc[0:2]
cols = [c for c in personnel.columns if 'fringe' in c.lower() or 'salary' in c.lower() or 'months' in c.lower() or 'effort' in c.lower()]
personnel = personnel[cols]
newColNames = {}
for i, col in enumerate(personnel.keys()):
    newColNames[col] = col.replace('Requested s','S').replace(' benefits','').replace(' Effort','')
personnel = personnel.rename(columns=newColNames)
personnelYear = []
budgetDF = 0
for i in range(1,6):
    cols = [c for c in personnel.columns if f'Y{i}' in c]
    tempDF = personnel[cols]
    newColNames = {}
    for col in cols:
        newColNames[col] = col.replace(f'Y{i} ','PI ')

    tempDF = tempDF.rename(columns=newColNames)
    postDocDF = tempDF.iloc[1]
    newDF = tempDF.drop(1)
    for col in tempDF.keys():
        postdocColName = col.replace('PI', 'Postdoc')
        newDF[postdocColName] = postDocDF[col]
        newDF = newDF.rename(index={0:f'Period {i}'})
        
    personnelYear.append(newDF)
personnelDF = pd.concat(personnelYear)
personnelDF = personnelDF.round(2)

format_mapping={}
for col in personnelDF:
    if 'month' in col.lower():
        format_mapping[col] = '{:,.1f}'
    else:
        format_mapping[col] = '${:,.0f}'
        
personnelDF.loc['Total']= personnelDF.sum(axis=0)
personnelDF = personnelDF.round(2)

for key, value in format_mapping.items():
    personnelDF[key] = personnelDF[key].apply(value.format)
personnelDF

,PI Months,PI Salary,PI Fringe,PI Total,Postdoc Months,Postdoc Salary,Postdoc Fringe,Postdoc Total
Period 1,9.0,"$136,246","$40,874","$177,120",12.5,"$77,778","$19,055","$96,833"
Period 2,7.0,"$122,733","$36,820","$159,553",18.0,"$116,541","$28,553","$145,094"
Period 3,6.0,"$108,992","$32,698","$141,690",29.5,"$198,431","$48,615","$247,046"
Period 4,6.0,"$112,924","$33,877","$146,801",17.0,"$118,806","$29,108","$147,914"
Period 5,6.0,"$117,002","$35,100","$152,102",12.0,"$87,132","$21,347","$108,479"
Total,34.0,"$597,897","$179,369","$777,266",89.0,"$598,688","$146,678","$745,366"


In [185]:
personnelLatex = personnelDF.to_latex()
personnelLatex = personnelLatex.replace('PI ', '').replace('Postdoc ', '')
personnelLatex = personnelLatex.replace('Overheads', 'Fringe').replace('Unloaded', 'Salary')
personnelLatex = personnelLatex.replace('\nTotal ', '\n\\hline\nTotal ')

# This can be done better...
personnelLatex = personnelLatex.replace('toprule\n', 'toprule\n & \\multicolumn{4}{c|}{Scientific} & \\multicolumn{4}{c}{Postdoctoral} \\\\\n')
personnelLatex = personnelLatex.replace('\\toprule', '\\hline').replace('\\midrule', '\\hline').replace('\\bottomrule', '\\hline')
# Again, this can be done more elegantly by counting the number of columns fol each type of pelsonnel. Maybe tomorrow...
personnelLatex = personnelLatex.replace('lllllllll', 'lrrrr|rrrr')
print(personnelLatex)
with open('personnelBudget.tex', 'w') as personnelF:  
    personnelF.write(personnelLatex)

\begin{tabular}{lrrrr|rrrr}
\hline
 & \multicolumn{4}{c|}{Scientific} & \multicolumn{4}{c}{Postdoctoral} \\
{} & Months & Salary & Fringe &  Total & Months & Salary & Fringe & Total \\
\hline
Period 1 &       9.0 &  \$136,246 &   \$40,874 &  \$177,120 &           12.5 &        \$77,778 &        \$19,055 &       \$96,833 \\
Period 2 &       7.0 &  \$122,733 &   \$36,820 &  \$159,553 &           18.0 &       \$116,541 &        \$28,553 &      \$145,094 \\
Period 3 &       6.0 &  \$108,992 &   \$32,698 &  \$141,690 &           29.5 &       \$198,431 &        \$48,615 &      \$247,046 \\
Period 4 &       6.0 &  \$112,924 &   \$33,877 &  \$146,801 &           17.0 &       \$118,806 &        \$29,108 &      \$147,914 \\
Period 5 &       6.0 &  \$117,002 &   \$35,100 &  \$152,102 &           12.0 &        \$87,132 &        \$21,347 &      \$108,479 \\
\hline
Total    &      34.0 &  \$597,897 &  \$179,369 &  \$777,266 &           89.0 &       \$598,688 &       \$146,678 &      \$745,366 \\
\hl

In [93]:
# Now let's get the travel costs.
cols = [col for col in df.keys() if 'Travel' in col]
travelCosts = df[cols].iloc[[4,6,8]]
newColNames = {}
for i, col in enumerate(travelCosts.keys()):
    newColNames[col] = col.replace(f'Y{i+1} Travel',f'Budget period {i+1}')
travelCosts = travelCosts.rename(columns=newColNames).rename(index={4:'Domestic trips', 5:'Domestic travel funds', 6:'International trips', 7:'International travel funds', 8:'Travel Funds'}).transpose()
travelCosts = travelCosts.round(2)
for col in travelCosts.columns:
    if col == 'Travel Funds':
        continue
    travelCosts[col] = travelCosts[col].map('{:,.0f}'.format)
travelCosts['Travel Funds'] = travelCosts['Travel Funds'].map('${:,.0f}'.format)

travelLatex = travelCosts.to_latex()

travelLatex = travelLatex.replace('\\toprule', '\\hline').replace('\\midrule', '\\hline').replace('\\bottomrule', '\\hline')
travelLatex = travelLatex.replace('lrrl', 'lrrr')
print(travelLatex)
with open('travelBudget.tex', 'w') as travelF:  
    travelF.write(travelLatex)


\begin{tabular}{llll}
\hline
{} & Domestic trips & International trips & Travel Funds \\
\hline
Budget period 1 &              0 &                   2 &       \$6,000 \\
Budget period 2 &              2 &                   5 &      \$18,000 \\
Budget period 3 &              3 &                   6 &      \$22,500 \\
Budget period 4 &              0 &                   7 &      \$21,000 \\
Budget period 5 &              3 &                   4 &      \$16,500 \\
\hline
\end{tabular}



In [120]:
# Now for total costs
cols = [col for col in df.keys() if 'Indirects' in col or 'Total' in col or 'Directs' in col or 'Travel' in col or 'Material' in col]
totals = df[cols]
totals = totals.iloc[[2,3,8,9,10,11]]
absTotals = totals.sum(axis=1)
print(absTotals)
outStr = f'The total direct costs are \${absTotals.iloc[3]:,.2f} of which \${absTotals.iloc[0]:,.2f} is in salaries, \${absTotals.iloc[1]:,.2f} is for equipment, and \${absTotals.iloc[2]:,.2f} is for travel.'
print(outStr)
with open('directs.tex', 'w') as directsF:  
    directsF.write(outStr)


2     1522632.0
3        8524.0
8       84000.0
9     1615156.0
10     884844.0
11    2500000.0
dtype: float64
The total direct costs are \$1,615,156.00 of which \$1,522,632.00 is in salaries, \$8,524.00 is for equipment, and \$84,000.00 is for travel.


In [128]:
cols = [col for col in df.keys() if 'Indirects' in col]
indirects = df[cols]
newColNames = {}
for i, col in enumerate(indirects.keys()):
    newColNames[col] = col.replace(f'Y{i+1} Indirects',f'Budget period {i+1}')
indirects = pd.DataFrame(indirects.rename(columns=newColNames).iloc[10])
indirectsFormatted = indirects.round(2)
for col in indirectsFormatted.keys():
    indirectsFormatted[col] = indirectsFormatted[col].map('${:,.2f}'.format)
indirectsLatex = indirectsFormatted.to_latex()

indirectsLatex = indirectsLatex.replace('\\toprule', '\\hline').replace('\\midrule', '\\hline').replace('\\bottomrule', '\\hline')
indirectsLatex = indirectsLatex.replace('lll', 'lrr')
indirectsLatex = indirectsLatex.replace('Total', '\\hline\nTotal')
print(indirectsLatex)
with open('indirects.tex', 'w') as indirectsF:  
    indirectsF.write(indirectsLatex)


\begin{tabular}{ll}
\hline
{} &           10 \\
\hline
Budget period 1 &  \$157,068.00 \\
Budget period 2 &  \$177,104.00 \\
Budget period 3 &  \$226,662.00 \\
Budget period 4 &  \$172,242.00 \\
Budget period 5 &  \$151,768.00 \\
\hline
\end{tabular}



In [173]:
cols = [col for col in df.keys() if 'Directs' in col]
directs = df[cols].iloc[9]
newIndexNames = {}
for i, col in enumerate(cols):
    newIndexNames[col] = f'Budget period {i+1}'
directs = directs.rename(newIndexNames)
cols = [col for col in df.keys() if 'Indirects' in col]
indirects = df[cols].iloc[10]
newIndexNames = {}
for i, col in enumerate(cols):
    newIndexNames[col] = f'Budget period {i+1}'
indirects = indirects.rename(index=newIndexNames)
cols = [col for col in df.keys() if 'Total Requested' in col]
totals = df[cols].iloc[11]
newIndexNames = {}
index = []
for i, col in enumerate(cols):
    newIndexNames[col] = f'Budget period {i+1}'
    index.append(newIndexNames[col])
totals = totals.rename(index=newIndexNames)
summaryDF = pd.concat([directs, indirects, totals], axis=1)
summaryDF = summaryDF.rename(columns={9:'Directs', 10:'Indirects', 11:'Total cost'})
summaryDF.loc['Total'] = summaryDF.sum(axis=0)
print(summaryDF)

totalCostsFormatted = summaryDF.round(2)
for col in totalCostsFormatted.keys():
    totalCostsFormatted[col] = totalCostsFormatted[col].map('${:,.2f}'.format)
totalCostsLatex = totalCostsFormatted.to_latex()

totalCostsLatex = totalCostsLatex.replace('\\toprule', '\\hline').replace('\\midrule', '\\hline').replace('\\bottomrule', '\\hline')
totalCostsLatex = totalCostsLatex.replace('lrrl', 'lrrr')
totalCostsLatex = totalCostsLatex.replace('\nTotal', '\\hline\nTotal')
print(totalCostsLatex)
with open('totalCosts.tex', 'w') as totalCostsF:  
    totalCostsF.write(totalCostsLatex)
outStr = f'The total cost of this proposal is {totalCostsFormatted["Total cost"].loc["Total"]}. '.replace("$","\\$")
with open('totalCostsText.tex', 'w') as totalCostsF:  
    totalCostsF.write(outStr)

                   Directs  Indirects  Total cost
Budget period 1   285353.0   157068.0    442421.0
Budget period 2   322647.0   177104.0    499751.0
Budget period 3   414360.0   226662.0    641022.0
Budget period 4   315715.0   172242.0    487957.0
Budget period 5   277081.0   151768.0    428849.0
Total            1615156.0   884844.0   2500000.0
\begin{tabular}{llll}
\hline
{} &        Directs &    Indirects &     Total cost \\
\hline
Budget period 1 &    \$285,353.00 &  \$157,068.00 &    \$442,421.00 \\
Budget period 2 &    \$322,647.00 &  \$177,104.00 &    \$499,751.00 \\
Budget period 3 &    \$414,360.00 &  \$226,662.00 &    \$641,022.00 \\
Budget period 4 &    \$315,715.00 &  \$172,242.00 &    \$487,957.00 \\
Budget period 5 &    \$277,081.00 &  \$151,768.00 &    \$428,849.00 \\\hline
Total           &  \$1,615,156.00 &  \$884,844.00 &  \$2,500,000.00 \\
\hline
\end{tabular}

